In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pickle import Pickler, Unpickler
import Geohash

In [2]:
#Importando Dataset
df = Unpickler(open('./df_rdo_categ.sav', 'rb')).load()

In [3]:
df.shape

(1766045, 24)

In [4]:
df = df.drop(columns=['LATITUDE', 'LONGITUDE'])
df.head()

,IDADE_PESSOA,GEOHASH,ANO,MES,DIA,HORA,MINUTOS,RUBRICA_Estupro,RUBRICA_Furto,RUBRICA_Homicídio,...,SEXO_PESSOA_,SEXO_PESSOA_F,SEXO_PESSOA_I,SEXO_PESSOA_M,COR_CUTIS_Amarela,COR_CUTIS_Branca,COR_CUTIS_Outros,COR_CUTIS_Parda,COR_CUTIS_Preta,COR_CUTIS_Vermelha
749,28,6gyf1,2011,04,01,15,10,0,1,0,...,0,1,0,0,0,1,0,0,0,0
750,61,6gyf4,2011,10,01,08,10,0,1,0,...,0,0,0,1,0,1,0,0,0,0
751,20,6gyf4,2011,10,01,08,30,0,0,0,...,0,1,0,0,0,1,0,0,0,0
752,42,6gyf4,2011,11,01,10,00,0,1,0,...,0,0,0,1,0,1,0,0,0,0
753,37,6gyf4,2011,12,01,06,40,0,1,0,...,0,0,0,1,0,0,0,1,0,0


In [5]:
X = df.drop(columns=['GEOHASH']).values
y = df[['GEOHASH']]
y = pd.get_dummies(y, columns = ['GEOHASH'])
y = y.values

In [6]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

/home/marco/dl_env/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/marco/dl_env/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/marco/dl_env/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## Construindo classificador

In [7]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [8]:
len(df['GEOHASH'].unique())

282

In [10]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 21))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 282, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [11]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
print(accuracies.mean())
print(accuracies.std())

/home/marco/dl_env/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/backend/utils.py:55: UserWarning: Failed to kill subprocesses on this platform. Pleaseinstall psutil: https://github.com/giampaolo/psutil
  warnings.warn("Failed to kill subprocesses on this platform. Please"
/home/marco/dl_env/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/backend/utils.py:55: UserWarning: Failed to kill subprocesses on this platform. Pleaseinstall psutil: https://github.com/giampaolo/psutil
  warnings.warn("Failed to kill subprocesses on this platform. Please"
/home/marco/dl_env/lib/python3.6/site-packages/sklearn/externals/joblib/externals/loky/backend/utils.py:55: UserWarning: Failed to kill subprocesses on this platform. Pleaseinstall psutil: https://github.com/giampaolo/psutil
  warnings.warn("Failed to kill subprocesses on this platform. Please"


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker. The exit codes of the workers are {SIGKILL(-9)}